In [2]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount("/content/grive")

Mounted at /content/grive


In [4]:
bank_data = pd.read_csv('/content/grive/MyDrive/BankCaseStudyData.csv')

In [5]:
bank_data.columns

Index(['Account_Number', 'Account_Type', 'Final_Decision',
       'Current_Delinquency_status', 'Application_Date', 'Application_Score',
       'Cheque_Card_Flag', 'Existing_Customer_Flag', 'Gross_Annual_Income',
       'Home_Telephone_Number', 'Insurance_Required', 'Loan_Amount',
       'Loan_Payment_Frequency', 'Loan_Payment_Method', 'Marital_Status',
       'Number_of_Dependants', 'Number_of_Payments', 'Occupation_Code',
       'Promotion_Type', 'Residential_Status', 'Time_at_Address',
       'Time_in_Employment', 'Time_with_Bank', 'Weight_Factor', 'GB_Flag',
       'Age_of_Applicant', 'Application_Month', 'Bureau_Score',
       'SP_ER_Reference', 'SP_Number_Of_Searches_L6M', 'SP_Number_of_CCJs',
       'loan_to_income', 'split'],
      dtype='object')

TARGET VARIABLE

In [6]:
bank_data['Target'] = (
    np.select(
        condlist = [
                    ((bank_data['Current_Delinquency_status'].isna()) & (bank_data['Final_Decision'] == 'Accept' )),
                    ((bank_data['Current_Delinquency_status'].isna()) & (bank_data['Final_Decision'] == 'Decline')),
                    bank_data['Current_Delinquency_status'] < 2,
                    bank_data['Current_Delinquency_status'] == 2,
                    bank_data['Current_Delinquency_status'] > 2
        ],
        choicelist = ['NTU', 'Rejects' , 'Good', 'Intermediate', 'Bad']
    )
)

In [7]:
bank_data = bank_data[((bank_data['Target'] == 'Good') | (bank_data['Target'] == 'Bad'))]

In [8]:
bank_data['Num_Target'] = np.where(bank_data['Target'] == 'Good', 1, 0)

In [9]:
bank_data.shape

(17968, 35)

PREDICTIVE FEATURES

Bureau Score

In [10]:
bank_data['Bureau_Score'].value_counts(dropna=False)

941     633
917     621
1013    425
970     374
972     369
       ... 
658       1
684       1
636       1
1035      1
672       1
Name: Bureau_Score, Length: 357, dtype: int64

In [11]:
bank_data['Bureau_Score'].describe()

count    17968.000000
mean       933.011520
std         68.032139
min        636.000000
25%        888.000000
50%        941.000000
75%        983.000000
max       1058.000000
Name: Bureau_Score, dtype: float64

In [12]:
bank_data['Bureau_Score_classing'] = pd.cut(bank_data['Bureau_Score'], [636, 830, 935, 970, 995, 1020, pd.np.inf], right=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [13]:
bank_data['Bureau_Score_classing'].value_counts(dropna=False).sort_index()

[636.0, 830.0)     1502
[830.0, 935.0)     6486
[935.0, 970.0)     3227
[970.0, 995.0)     2985
[995.0, 1020.0)    2529
[1020.0, inf)      1239
Name: Bureau_Score_classing, dtype: int64

In [14]:
bank_data.groupby(['Bureau_Score_classing'])['Num_Target'].mean()

Bureau_Score_classing
[636.0, 830.0)     0.825566
[830.0, 935.0)     0.912427
[935.0, 970.0)     0.959715
[970.0, 995.0)     0.969179
[995.0, 1020.0)    0.977066
[1020.0, inf)      0.983858
Name: Num_Target, dtype: float64

In [15]:
pd.crosstab(bank_data['Bureau_Score_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Bureau_Score_classing,,,
"[636.0, 830.0)",262,1240,1502
"[830.0, 935.0)",568,5918,6486
"[935.0, 970.0)",130,3097,3227
"[970.0, 995.0)",92,2893,2985
"[995.0, 1020.0)",58,2471,2529
"[1020.0, inf)",20,1219,1239
All,1130,16838,17968


Cheque Card Flag

In [16]:
bank_data['Cheque_Card_Flag_classing'] = np.where(bank_data['Cheque_Card_Flag'] == "Y", 'Y', 'N')
bank_data['Cheque_Card_Flag_classing'].value_counts(dropna=False)

Y    12235
N     5733
Name: Cheque_Card_Flag_classing, dtype: int64

In [17]:
bank_data.groupby(['Cheque_Card_Flag_classing'])['Num_Target'].mean()

Cheque_Card_Flag_classing
N    0.894122
Y    0.957254
Name: Num_Target, dtype: float64

In [18]:
pd.crosstab(bank_data['Cheque_Card_Flag_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Cheque_Card_Flag_classing,,,
N,607,5126,5733
Y,523,11712,12235
All,1130,16838,17968


Occupation Code

In [19]:
bank_data['Occupation_Code_classing'] = np.where(bank_data['Occupation_Code'] == "M" , 'M', 'P|B|O')
bank_data['Occupation_Code_classing'].value_counts(dropna=False)

P|B|O    9702
M        8266
Name: Occupation_Code_classing, dtype: int64

In [20]:
bank_data.groupby(['Occupation_Code_classing'])['Num_Target'].mean()

Occupation_Code_classing
M        0.921849
P|B|O    0.950113
Name: Num_Target, dtype: float64

In [21]:
pd.crosstab(bank_data['Occupation_Code_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Occupation_Code_classing,,,
M,646,7620,8266
P|B|O,484,9218,9702
All,1130,16838,17968


Residential Status

In [23]:
bank_data['Residential_Status_classing'] = np.where(bank_data['Residential_Status'] == "H", 'H', 'L|O|T')
bank_data['Residential_Status_classing'].value_counts(dropna=False).sort_index()

H        12202
L|O|T     5766
Name: Residential_Status_classing, dtype: int64

In [24]:
bank_data.groupby(['Residential_Status_classing'])['Num_Target'].mean()

Residential_Status_classing
H        0.952713
L|O|T    0.904093
Name: Num_Target, dtype: float64

In [25]:
pd.crosstab(bank_data['Residential_Status_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Residential_Status_classing,,,
H,577,11625,12202
L|O|T,553,5213,5766
All,1130,16838,17968


Loan Payment Frequency

In [29]:
bank_data['Loan_Payment_Frequency_classing'] = np.where(bank_data['Loan_Payment_Frequency'] == "M", 'M', 'F|W|X')
bank_data['Loan_Payment_Frequency_classing'].value_counts(dropna=False)

M        9584
F|W|X    8384
Name: Loan_Payment_Frequency_classing, dtype: int64

In [30]:
bank_data.groupby(['Loan_Payment_Frequency_classing'])['Num_Target'].mean()

Loan_Payment_Frequency_classing
F|W|X    0.918177
M        0.953673
Name: Num_Target, dtype: float64

In [31]:
pd.crosstab(bank_data['Loan_Payment_Frequency_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Loan_Payment_Frequency_classing,,,
F|W|X,686,7698,8384
M,444,9140,9584
All,1130,16838,17968


Loan Payment Method

In [32]:
bank_data['Loan_Payment_Method_classing'] = np.where(bank_data['Loan_Payment_Method'] == "B", 'B', 'Q|S|X')
bank_data['Loan_Payment_Method_classing'].value_counts(dropna=False)

B        12359
Q|S|X     5609
Name: Loan_Payment_Method_classing, dtype: int64

In [33]:
bank_data.groupby(['Loan_Payment_Method_classing'])['Num_Target'].mean()

Loan_Payment_Method_classing
B        0.948135
Q|S|X    0.912819
Name: Num_Target, dtype: float64

In [34]:
pd.crosstab(bank_data['Loan_Payment_Method_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Loan_Payment_Method_classing,,,
B,641,11718,12359
Q|S|X,489,5120,5609
All,1130,16838,17968


Insurance Required

In [35]:
bank_data['Insurance_Required_classing'] = np.where(bank_data['Insurance_Required'] == "N", 'N', 'Y|Nan')
bank_data.groupby(['Insurance_Required_classing'])['Num_Target'].mean()

Insurance_Required_classing
N        0.951140
Y|Nan    0.917479
Name: Num_Target, dtype: float64

In [36]:
bank_data['Insurance_Required_classing'].value_counts(dropna=False)

N        10479
Y|Nan     7489
Name: Insurance_Required_classing, dtype: int64

In [37]:
pd.crosstab(bank_data['Insurance_Required_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Insurance_Required_classing,,,
N,512,9967,10479
Y|Nan,618,6871,7489
All,1130,16838,17968


Marital Status

In [26]:
bank_data['Marital_Status_classing'] = np.where(bank_data['Marital_Status'] == "M", 'M' , 'Other')
bank_data.groupby(['Marital_Status_classing'])['Num_Target'].mean()

Marital_Status_classing
M        0.948336
Other    0.920118
Name: Num_Target, dtype: float64

In [27]:
bank_data['Marital_Status_classing'].value_counts(dropna=False)

M        10820
Other     7148
Name: Marital_Status_classing, dtype: int64

In [28]:
pd.crosstab(bank_data['Marital_Status_classing'],bank_data['Target'], margins="Total")

Target,Bad,Good,All
Marital_Status_classing,,,
M,559,10261,10820
Other,571,6577,7148
All,1130,16838,17968


DUMMY VARIABLES

In [ ]:
bank_data_model = bank_data[['Target', 'Num_Target', 'Bureau_Score' , 'Bureau_Score_classing', 'Cheque_Card_Flag', 'Cheque_Card_Flag_classing', 'Occupation_Code','Occupation_Code_classing', 'Residential_Status','Residential_Status_classing', 'Loan_Payment_Frequency', 'Loan_Payment_Frequency_classing', 'Loan_Payment_Method','Loan_Payment_Method_classing','Insurance_Required','Insurance_Required_classing', 'Marital_Status', 'Marital_Status_classing']].copy()

In [ ]:
bank_data_model = pd.concat([bank_data_model,
                             pd.get_dummies(bank_data_model['Bureau_Score_classing'], prefix='Bureau_Score', dummy_na=False),
                             pd.get_dummies(bank_data_model['Cheque_Card_Flag_classing'], prefix='Cheque_Card_Flag', dummy_na=False),
                             pd.get_dummies(bank_data_model['Occupation_Code_classing'], prefix='Occupation_code', dummy_na=False),
                             pd.get_dummies(bank_data_model['Residential_Status_classing'], prefix='Residential_Status', dummy_na=False),
                             pd.get_dummies(bank_data_model['Loan_Payment_Frequency_classing'], prefix='Loan_Payment_Frequency', dummy_na=False),
                             pd.get_dummies(bank_data_model['Loan_Payment_Method_classing'], prefix='Loan_Payment_Method', dummy_na=False),
                             pd.get_dummies(bank_data_model['Insurance_Required_classing'], prefix='Insurance_Required', dummy_na=False),
                             pd.get_dummies(bank_data_model['Marital_Status_classing'], prefix='Marital_Status', dummy_na=False),],
                                                            axis=1, ignore_index=False, join='outer')

In [ ]:
bank_data_model.columns

Index(['Target', 'Num_Target', 'Bureau_Score', 'Bureau_Score_classing',
       'Cheque_Card_Flag', 'Cheque_Card_Flag_classing', 'Occupation_Code',
       'Occupation_Code_classing', 'Residential_Status',
       'Residential_Status_classing', 'Loan_Payment_Frequency',
       'Loan_Payment_Frequency_classing', 'Loan_Payment_Method',
       'Loan_Payment_Method_classing', 'Insurance_Required',
       'Insurance_Required_classing', 'Marital_Status',
       'Marital_Status_classing', 'Bureau_Score_[636.0, 830.0)',
       'Bureau_Score_[830.0, 935.0)', 'Bureau_Score_[935.0, 970.0)',
       'Bureau_Score_[970.0, 995.0)', 'Bureau_Score_[995.0, 1020.0)',
       'Bureau_Score_[1020.0, inf)', 'Cheque_Card_Flag_N',
       'Cheque_Card_Flag_Y', 'Occupation_code_M', 'Occupation_code_P|B|O',
       'Residential_Status_H', 'Residential_Status_L|O|T',
       'Loan_Payment_Frequency_F|W|X', 'Loan_Payment_Frequency_M',
       'Loan_Payment_Method_B', 'Loan_Payment_Method_Q|S|X',
       'Insurance_Require

Split data frama into training and test sets

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(bank_data_model,bank_data_model['Num_Target'], test_size=0.2,random_state=42)

In [ ]:
x_train['sample']='Training'
x_test['sample'] = 'Testing'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
bank_data_model = pd.concat([x_train,x_test], ignore_index=False, sort=False)
bank_data_model = bank_data_model.reset_index(drop=True)
bank_data_model.shape

(17968, 39)

In [ ]:
y = bank_data_model[bank_data_model['sample'] == "Training"]['Num_Target']
x = bank_data_model[bank_data_model['sample'] == "Training"][['Bureau_Score_[636.0, 830.0)',
                                                              'Bureau_Score_[830.0, 935.0)',
                                                              'Bureau_Score_[935.0, 970.0)',
                                                              'Bureau_Score_[970.0, 995.0)',
                                                              'Bureau_Score_[995.0, 1020.0)',
                                                              'Bureau_Score_[1020.0, inf)',
                                                              'Cheque_Card_Flag_N',
                                                              'Cheque_Card_Flag_Y',
                                                              'Occupation_code_P|B|O',
                                                              'Occupation_code_M',
                                                              'Residential_Status_H',
                                                              'Residential_Status_L|O|T',
                                                              'Loan_Payment_Frequency_F|W|X',
                                                              'Loan_Payment_Frequency_M',
                                                              'Loan_Payment_Method_B',
                                                              'Loan_Payment_Method_Q|S|X',
                                                              'Insurance_Required_N',
                                                              'Insurance_Required_Y|Nan',
                                                              'Marital_Status_M',
                                                              'Marital_Status_Other'
                                                              ]]

In [ ]:
bank_data_model['sample'].value_counts(dropna=False)

Training    14374
Testing      3594
Name: sample, dtype: int64

MODEL

In [ ]:
x = sm.tools.tools.add_constant(x)
model_1 = sm.GLM(y,x,family=sm.genmod.families.Binomial(link=sm.genmod.families.links.logit))
model_final = model_1.fit()
print(model_final.summary())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  


                 Generalized Linear Model Regression Results                  
Dep. Variable:             Num_Target   No. Observations:                14374
Model:                            GLM   Df Residuals:                    14361
Model Family:                Binomial   Df Model:                           12
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3195.1
Date:                Thu, 01 Jul 2021   Deviance:                       6390.3
Time:                        06:11:13   Pearson chi2:                 1.43e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

PREDICTION

In [ ]:
x['Prediction'] = model_final.predict()
x

,const,"Bureau_Score_[636.0, 830.0)","Bureau_Score_[830.0, 935.0)","Bureau_Score_[935.0, 970.0)","Bureau_Score_[970.0, 995.0)","Bureau_Score_[995.0, 1020.0)","Bureau_Score_[1020.0, inf)",Cheque_Card_Flag_N,Cheque_Card_Flag_Y,Occupation_code_P|B|O,Occupation_code_M,Residential_Status_H,Residential_Status_L|O|T,Loan_Payment_Frequency_F|W|X,Loan_Payment_Frequency_M,Loan_Payment_Method_B,Loan_Payment_Method_Q|S|X,Insurance_Required_N,Insurance_Required_Y|Nan,Marital_Status_M,Marital_Status_Other,Prediction
0,1.0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,1,0,1,0,0,1,0.980759
1,1.0,0,0,0,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,0.979749
2,1.0,0,0,0,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,0.979749
3,1.0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,1,0,1,0,0.942712
4,1.0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,0.969890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14369,1.0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0.895220
14370,1.0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0.933819
14371,1.0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0.896265
14372,1.0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,0.873072


In [ ]:
x.columns.values

array(['const', 'Bureau_Score_[636.0, 830.0)',
       'Bureau_Score_[830.0, 935.0)', 'Bureau_Score_[935.0, 970.0)',
       'Bureau_Score_[970.0, 995.0)', 'Bureau_Score_[995.0, 1020.0)',
       'Bureau_Score_[1020.0, inf)', 'Cheque_Card_Flag_N',
       'Cheque_Card_Flag_Y', 'Occupation_code_P|B|O', 'Occupation_code_M',
       'Residential_Status_H', 'Residential_Status_L|O|T',
       'Loan_Payment_Frequency_F|W|X', 'Loan_Payment_Frequency_M',
       'Loan_Payment_Method_B', 'Loan_Payment_Method_Q|S|X',
       'Insurance_Required_N', 'Insurance_Required_Y|Nan',
       'Marital_Status_M', 'Marital_Status_Other', 'Prediction'],
      dtype=object)

In [ ]:
model_coefficients = {
    "Bureau_Score_[636.0, 830.0)": -1.1097,
    "Bureau_Score_[830.0, 935.0)": -0.4672,
    "Bureau_Score_[935.0, 970.0)": 0.2633,
    "Bureau_Score_[970.0, 995.0)": 0.5034,
    "Bureau_Score_[995.0, 1020.0)": 0.6701,
    "Bureau_Score_[1020.0, inf)": 0.7523,
    "Cheque_Card_Flag_N": 0.1170,
    "Cheque_Card_Flag_Y": 0.4952,
    "Occupation_code_P|B|O": 0.2914,
    "Occupation_code_M": 0.3208,
    "Residential_Status_H": 0.3385,
    "Residential_Status_L|O|T": 0.2737,
    "Loan_Payment_Frequency_F|W|X": 0.2970,
    "Loan_Payment_Frequency_M": 0.3152,
    "Insurance_Required_N": 0.3679,
    "Insurance_Required_Y|Nan": 0.2443,
    "Loan_Payment_Method_B": 0.4676,
    "Loan_Payment_Method_Q|S|X": 0.1446,
    "Marital_Status_M": 0.3211,
    "Marital_Status_Other": 0.2911 
}

In [ ]:
def score(df, intercept, coefs):
  value_to_add = 0
  for name, coef in coefs.items():
    value_to_add += coef*(df[name])
  df["Score"] = round((intercept + value_to_add)*100.0)

In [ ]:
score(bank_data_model, 0.6122, model_coefficients)

SCORE DISTRIBUTION REPORT

In [ ]:
bank_data_model['Score']

0        393.0
1        388.0
2        388.0
3        280.0
4        347.0
         ...  
17963    265.0
17964    268.0
17965    216.0
17966    151.0
17967    274.0
Name: Score, Length: 17968, dtype: float64

In [ ]:
bank_data_model['Score'].value_counts(dropna=False).sort_index()

116.0     31
118.0     15
119.0    122
121.0     10
122.0     24
        ... 
393.0     72
394.0     65
396.0    884
397.0     58
399.0     66
Name: Score, Length: 237, dtype: int64

In [ ]:
bank_data_model['Score'].describe()

count    17968.000000
mean       292.399655
std         76.768152
min        116.000000
25%        227.000000
50%        303.000000
75%        362.000000
max        399.000000
Name: Score, dtype: float64

In [ ]:
bank_dev = bank_data_model[bank_data_model['sample'] == 'Training']
bank_val = bank_data_model[bank_data_model['sample'] == 'Testing' ]

In [ ]:
for i in range(0, 101, 5):
  print(np.percentile(bank_dev['Score'],i), ",")

116.0 ,
154.0 ,
193.0 ,
210.0 ,
218.0 ,
227.0 ,
237.0 ,
255.0 ,
266.0 ,
275.0 ,
304.0 ,
318.0 ,
335.0 ,
340.0 ,
354.0 ,
362.0 ,
371.0 ,
385.0 ,
388.0 ,
396.0 ,
399.0 ,


In [ ]:
SDR = pd.crosstab(pd.cut(bank_dev['Score'], bins= [
                                          116.0 ,
154.0 ,
193.0 ,
210.0 ,
218.0 ,
227.0 ,
237.0 ,
255.0 ,
266.0 ,
275.0 ,
304.0 ,
318.0 ,
335.0 ,
340.0 ,
354.0 ,
362.0 ,
371.0 ,
385.0 ,
388.0 ,
396.0 ,
399.0 ,
]), bank_dev["Target"])

In [ ]:
SDR['Bad_Rate'] = round((SDR['Bad']/(SDR['Bad'] + SDR['Good']))*100,2)
SDR['Good_Bad_Odds'] = round(SDR['Good']/SDR['Bad'], 2)
SDR['Total_%'] = round((SDR['Bad'] + SDR['Good'])/(SDR['Bad'].sum() + SDR['Good'].sum())*100,2)

In [ ]:
SDR

Target,Bad,Good,Bad_Rate,Good_Bad_Odds,Total_%
Score,,,,,
"(116.0, 154.0]",133,569,18.95,4.28,4.89
"(154.0, 193.0]",127,721,14.98,5.68,5.91
"(193.0, 210.0]",78,517,13.11,6.63,4.15
"(210.0, 218.0]",73,657,10.00,9.00,5.09
"(218.0, 227.0]",79,625,11.22,7.91,4.91
"(227.0, 237.0]",66,706,8.55,10.70,5.38
"(237.0, 255.0]",48,676,6.63,14.08,5.05
"(255.0, 266.0]",49,683,6.69,13.94,5.10
"(266.0, 275.0]",35,605,5.47,17.29,4.46


VALIDATION TEST

In [ ]:
validation_dev = pd.crosstab(pd.cut(bank_dev['Score'], bins=[
                                                            116.0 ,
154.0 ,
193.0 ,
210.0 ,
218.0 ,
227.0 ,
237.0 ,
255.0 ,
266.0 ,
275.0 ,
304.0 ,
318.0 ,
335.0 ,
340.0 ,
354.0 ,
362.0 ,
371.0 ,
385.0 ,
388.0 ,
396.0 ,
399.0 ,
]), bank_dev['Target'])

In [ ]:
validation_val = pd.crosstab(pd.cut(bank_val['Score'], bins=[
                                                         116.0 ,
154.0 ,
193.0 ,
210.0 ,
218.0 ,
227.0 ,
237.0 ,
255.0 ,
266.0 ,
275.0 ,
304.0 ,
318.0 ,
335.0 ,
340.0 ,
354.0 ,
362.0 ,
371.0 ,
385.0 ,
388.0 ,
396.0 ,
399.0 ,
]), bank_val['Target'])

In [ ]:
validation_total = pd.crosstab(pd.cut(bank_data_model['Score'], bins=[
                                                                    116.0 ,
154.0 ,
193.0 ,
210.0 ,
218.0 ,
227.0 ,
237.0 ,
255.0 ,
266.0 ,
275.0 ,
304.0 ,
318.0 ,
335.0 ,
340.0 ,
354.0 ,
362.0 ,
371.0 ,
385.0 ,
388.0 ,
396.0 ,
399.0 ,
]), bank_data_model['Target'])

In [ ]:
validation_total.reset_index(inplace=True)
validation_total.columns.name = 'index'
validation_dev.reset_index(inplace=True)
validation_dev.columns.name = 'index'
validation_val.reset_index(inplace=True)
validation_val.columns.name = 'index'

In [ ]:
validation_total['Total'] = validation_total['Bad'] + validation_total['Good']
validation_dev['Total'] = validation_dev['Bad'] + validation_dev['Good']
validation_val['Total'] = validation_val['Bad'] + validation_val['Good']

In [ ]:
df_combined = validation_dev.merge(right=validation_val, on='Score').merge(right=validation_total,on='Score')
df_combined

index,Score,Bad_x,Good_x,Total_x,Bad_y,Good_y,Total_y,Bad,Good,Total
0,"(116.0, 154.0]",133,569,702,35,146,181,168,715,883
1,"(154.0, 193.0]",127,721,848,20,203,223,147,924,1071
2,"(193.0, 210.0]",78,517,595,15,133,148,93,650,743
3,"(210.0, 218.0]",73,657,730,19,161,180,92,818,910
4,"(218.0, 227.0]",79,625,704,13,177,190,92,802,894
5,"(227.0, 237.0]",66,706,772,13,179,192,79,885,964
6,"(237.0, 255.0]",48,676,724,12,171,183,60,847,907
7,"(255.0, 266.0]",49,683,732,10,215,225,59,898,957
8,"(266.0, 275.0]",35,605,640,6,141,147,41,746,787
9,"(275.0, 304.0]",42,769,811,10,206,216,52,975,1027


In [ ]:
df_combined.columns

Index(['Score', 'Bad_x', 'Good_x', 'Total_x', 'Bad_y', 'Good_y', 'Total_y',
       'Bad', 'Good', 'Total'],
      dtype='object', name='index')

In [ ]:
rename = {
    'Bad_x': 'Bad_dev',
    'Bad_y': 'Bad_val',
    'Good_x': 'Good_dev',
    'Good_y': 'Good_val',
    'Bad': 'Bad_All',
    'Good': 'Good_All',
    'Total_x': 'Total_Dev',
    'Total_y': 'Total_val',
    'Total': 'Total_All'
}

In [ ]:
df_combined = df_combined.rename(columns = rename)
df_combined

index,Score,Bad_dev,Good_dev,Total_Dev,Bad_val,Good_val,Total_val,Bad_All,Good_All,Total_All
0,"(116.0, 154.0]",133,569,702,35,146,181,168,715,883
1,"(154.0, 193.0]",127,721,848,20,203,223,147,924,1071
2,"(193.0, 210.0]",78,517,595,15,133,148,93,650,743
3,"(210.0, 218.0]",73,657,730,19,161,180,92,818,910
4,"(218.0, 227.0]",79,625,704,13,177,190,92,802,894
5,"(227.0, 237.0]",66,706,772,13,179,192,79,885,964
6,"(237.0, 255.0]",48,676,724,12,171,183,60,847,907
7,"(255.0, 266.0]",49,683,732,10,215,225,59,898,957
8,"(266.0, 275.0]",35,605,640,6,141,147,41,746,787
9,"(275.0, 304.0]",42,769,811,10,206,216,52,975,1027


In [ ]:
df_combined.set_index('Score', inplace=True)

In [ ]:
df_combined = df_combined.cumsum()

In [ ]:
rename2 = {
    'Bad_dev': 'Cum_Bad_dev',
    'Good_dev': 'Cum_Good_dev',
    'Total_Dev': 'Cum_Total_dev',
    'Bad_val': 'Cum_Bad_val',
    'Good_val': 'Cum_Good_val',
    'Total_val': 'Cum_Total_val',
    'Bad_All': 'Cum_Bad_All',
    'Good_All': 'Cum_Good_All',
    'Total_All': 'Cum_Total_All'
}

In [ ]:
df_combined = df_combined.rename(columns=rename2)
df_combined

index,Cum_Bad_dev,Cum_Good_dev,Cum_Total_dev,Cum_Bad_val,Cum_Good_val,Cum_Total_val,Cum_Bad_All,Cum_Good_All,Cum_Total_All
Score,,,,,,,,,
"(116.0, 154.0]",133,569,702,35,146,181,168,715,883
"(154.0, 193.0]",260,1290,1550,55,349,404,315,1639,1954
"(193.0, 210.0]",338,1807,2145,70,482,552,408,2289,2697
"(210.0, 218.0]",411,2464,2875,89,643,732,500,3107,3607
"(218.0, 227.0]",490,3089,3579,102,820,922,592,3909,4501
"(227.0, 237.0]",556,3795,4351,115,999,1114,671,4794,5465
"(237.0, 255.0]",604,4471,5075,127,1170,1297,731,5641,6372
"(255.0, 266.0]",653,5154,5807,137,1385,1522,790,6539,7329
"(266.0, 275.0]",688,5759,6447,143,1526,1669,831,7285,8116


In [ ]:
df_combined['Cum_%_Bad_dev'] = round(df_combined['Cum_Bad_dev']/df_combined.iloc[-1,0]*100,2)
df_combined['Cum_%_Bad_val'] = round(df_combined['Cum_Bad_val']/df_combined.iloc[-1,3]*100,2)
df_combined['Cum_%_Bad_All'] = round(df_combined['Cum_Bad_All']/df_combined.iloc[-1,6]*100,2)

In [ ]:
df_combined['Cum_%_Good_dev'] = round(df_combined['Cum_Good_dev']/df_combined.iloc[-1,1]*100,2)
df_combined['Cum_%_Good_val'] = round(df_combined['Cum_Good_val']/df_combined.iloc[-1,4]*100,2)
df_combined['Cum_%_Good_All'] = round(df_combined['Cum_Good_All']/df_combined.iloc[-1,7]*100,2)

In [ ]:
df_combined['Cum_%_Total_dev'] = round(df_combined['Cum_Total_dev']/df_combined.iloc[-1,2]*100,2)
df_combined['Cum_%_Total_val'] = round(df_combined['Cum_Total_val']/df_combined.iloc[-1,5]*100,2)
df_combined['Cum_%_Total_All'] = round(df_combined['Cum_Total_All']/df_combined.iloc[-1,8]*100,2)

In [ ]:
print(df_combined['Cum_%_Bad_All'])

Score
(116.0, 154.0]     14.93
(154.0, 193.0]     28.00
(193.0, 210.0]     36.27
(210.0, 218.0]     44.44
(218.0, 227.0]     52.62
(227.0, 237.0]     59.64
(237.0, 255.0]     64.98
(255.0, 266.0]     70.22
(266.0, 275.0]     73.87
(275.0, 304.0]     78.49
(304.0, 318.0]     81.07
(318.0, 335.0]     84.71
(335.0, 340.0]     88.00
(340.0, 354.0]     89.87
(354.0, 362.0]     92.80
(362.0, 371.0]     93.69
(371.0, 385.0]     95.82
(385.0, 388.0]     97.51
(388.0, 396.0]    100.00
(396.0, 399.0]    100.00
Name: Cum_%_Bad_All, dtype: float64


In [ ]:
print(df_combined['Cum_%_Good_All'])

Score
(116.0, 154.0]      4.25
(154.0, 193.0]      9.75
(193.0, 210.0]     13.62
(210.0, 218.0]     18.48
(218.0, 227.0]     23.25
(227.0, 237.0]     28.52
(237.0, 255.0]     33.55
(255.0, 266.0]     38.89
(266.0, 275.0]     43.33
(275.0, 304.0]     49.13
(304.0, 318.0]     53.59
(318.0, 335.0]     59.44
(335.0, 340.0]     64.06
(340.0, 354.0]     68.99
(354.0, 362.0]     75.65
(362.0, 371.0]     79.65
(371.0, 385.0]     85.24
(385.0, 388.0]     91.13
(388.0, 396.0]     99.26
(396.0, 399.0]    100.00
Name: Cum_%_Good_All, dtype: float64


In [ ]:
df_combined['dev_val_cum_%_diff_bad'] = abs(df_combined['Cum_%_Bad_dev'] - df_combined['Cum_%_Bad_val'])
df_combined['dev_all_cum_%_diff_bad'] = abs(df_combined['Cum_%_Bad_dev'] - df_combined['Cum_%_Bad_All'])
df_combined['val_all_cum_%_diff_bad'] = abs(df_combined['Cum_%_Bad_val'] -  df_combined['Cum_%_Bad_All'])

In [ ]:
df_combined['dev_val_cum_%_diff_good'] = abs(df_combined['Cum_%_Good_dev'] - df_combined['Cum_%_Good_val'])
df_combined['dev_all_cum_%_diff_good'] = abs(df_combined['Cum_%_Good_dev'] - df_combined['Cum_%_Good_All'])
df_combined['val_all_cum_%_diff_good'] = abs(df_combined['Cum_%_Good_val'] - df_combined['Cum_%_Good_All'])

In [ ]:
df_combined['dev_val_cum_%_diff_total'] = abs(df_combined['Cum_%_Total_dev'] - df_combined['Cum_%_Total_val'])
df_combined['dev_all_cum_%_diff_total'] = abs(df_combined['Cum_%_Total_dev'] - df_combined['Cum_%_Total_All'])
df_combined['val_all_cum_%_diff_total'] = abs(df_combined['Cum_%_Total_val'] - df_combined['Cum_%_Total_All'])

KS TEST

In [ ]:
KS_dev_val_bad = round(1.358*np.sqrt((df_combined.iloc[-1,0] + df_combined.iloc[-1,3])/(df_combined.iloc[-1,0]*df_combined.iloc[-1,3]))*100,2)
KS_dev_all_bad = round(1.358*np.sqrt((df_combined.iloc[-1,0] + df_combined.iloc[-1,6])/(df_combined.iloc[-1,0]*df_combined.iloc[-1,6]))*100,2)
KS_val_all_bad = round(1.358*np.sqrt((df_combined.iloc[-1,3] + df_combined.iloc[-1,6])/(df_combined.iloc[-1,3]*df_combined.iloc[-1,6]))*100,2)

KS_dev_val_good = round(1.358*np.sqrt((df_combined.iloc[-1,1] + df_combined.iloc[-1,4])/(df_combined.iloc[-1,1]*df_combined.iloc[-1,4]))*100,2)
KS_dev_all_good = round(1.358*np.sqrt((df_combined.iloc[-1,1] + df_combined.iloc[-1,7])/(df_combined.iloc[-1,1]*df_combined.iloc[-1,7]))*100,2)
KS_val_all_good = round(1.358*np.sqrt((df_combined.iloc[-1,4] + df_combined.iloc[-1,7])/(df_combined.iloc[-1,4]*df_combined.iloc[-1,7]))*100,2)

KS_dev_val_total = round(1.358*np.sqrt((df_combined.iloc[-1,2] + df_combined.iloc[-1,5])/(df_combined.iloc[-1,2]*df_combined.iloc[-1,5]))*100,2)
KS_dev_all_total = round(1.358*np.sqrt((df_combined.iloc[-1,2] + df_combined.iloc[-1,8])/(df_combined.iloc[-1,2]*df_combined.iloc[-1,8]))*100,2)
KS_val_all_total = round(1.358*np.sqrt((df_combined.iloc[-1,5] + df_combined.iloc[-1,8])/(df_combined.iloc[-1,5]*df_combined.iloc[-1,8]))*100,2)

In [ ]:
test = {'dev_val_cum_%_diff_bad': KS_dev_val_bad,
        'dev_all_cum_%_diff_bad': KS_dev_all_bad,
        'val_all_cum_%_diff_bad': KS_val_all_bad, 
        
        'dev_val_cum_%_diff_good':KS_dev_val_good, 
        'dev_all_cum_%_diff_good': KS_dev_all_good, 
        'val_all_cum_%_diff_good': KS_val_all_good,
        
        'dev_val_cum_%_diff_total': KS_dev_val_total, 
        'dev_all_cum_%_diff_total':KS_dev_all_total, 
        'val_all_cum_%_diff_total':KS_val_all_total}

In [ ]:
validation = {}
for key, value in test.items():
    if df_combined[key].max() < value:
        validation[key] = 'Valid'
    else:
        validation[key] = 'Invalid'

In [ ]:
for key, value in validation.items():
    print(f'{key} is {value}!')

dev_val_cum_%_diff_bad is Valid!
dev_all_cum_%_diff_bad is Valid!
val_all_cum_%_diff_bad is Valid!
dev_val_cum_%_diff_good is Valid!
dev_all_cum_%_diff_good is Valid!
val_all_cum_%_diff_good is Valid!
dev_val_cum_%_diff_total is Valid!
dev_all_cum_%_diff_total is Valid!
val_all_cum_%_diff_total is Valid!
